### Conversational AI (Chatbot)

In [1]:
import os
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai
from IPython.display import Markdown, display, clear_output

In [2]:
load_dotenv()

google_api_key = os.getenv('GEMINI_API_KEY')

In [3]:
genai.configure(api_key=google_api_key)

In [4]:
system_message = "You are a helpful assistant"

In [5]:
def message_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",  # or "gemini-1.5-pro"
        system_instruction=system_message
    )
    
    response = model.generate_content(prompt)
    return response.text


In [6]:
view = gr.Interface(
    fn=message_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Streaming Output

In [7]:

system_message = "You are a helpful assistant that responds in markdown"  # Optional system instruction

def stream_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )

    stream = model.generate_content(prompt, stream=True)

    result = ""
    for chunk in stream:
        if chunk.text:
            result += chunk.text
            yield result


In [8]:
for partial in stream_gemini("Explain quantum computing in simple terms."):
    clear_output(wait=True)
    display(Markdown(partial))

Okay, let's break down quantum computing in simpler terms:

**Think of it like this:  Classical computers use bits, which are like light switches that are either on (1) or off (0). Quantum computers use *qubits*, which are like dimmer switches that can be both on AND off *at the same time***.

Here's a more detailed breakdown:

*   **Classical Bits:**  Imagine a light switch. It can be either ON (representing 1) or OFF (representing 0).  Everything your current computer does is based on manipulating these 0s and 1s.

*   **Quantum Bits (Qubits):** Now, imagine a dimmer switch instead of a simple on/off switch.  A qubit can be ON (1), OFF (0), or *anywhere in between*.  It exists in a combination of both states *simultaneously*. This "in-between" state is called **superposition**. Think of it like a coin spinning in the air – it's neither heads nor tails until it lands.

*   **Superposition:** The ability of a qubit to be in multiple states at once (0 and 1 simultaneously) is called superposition. This is a key difference from classical bits.  This allows quantum computers to explore many possibilities at the same time.

*   **Entanglement:**  Imagine you have two of these "dimmer switch" qubits.  Entanglement is when these two qubits become linked together in a special way.  If you measure the state of one entangled qubit, you instantly know the state of the other, no matter how far apart they are.  Einstein called it "spooky action at a distance."  Entanglement helps quantum computers perform calculations in a coordinated way.

*   **How Quantum Computers Work (Simplified):**

    1.  **Initialization:**  You set up a bunch of qubits in a specific superposition.
    2.  **Manipulation:**  You apply a series of quantum operations (like turning the dimmer switch a certain amount) to the qubits. These operations are carefully designed to perform a specific calculation. Because of superposition and entanglement, all the qubits are working together to explore many possibilities at once.
    3.  **Measurement:**  Finally, you "measure" the state of the qubits. When you measure, the superposition collapses, and each qubit settles into a definite 0 or 1 state.  The result of the measurement gives you the answer to the calculation.

*   **Why is this powerful?**  Because quantum computers can explore many possibilities simultaneously, they can potentially solve certain types of problems much faster than classical computers.  These are problems where trying every possibility one by one would take a classical computer an impossibly long time.

*   **What are Quantum Computers Good For?**

    *   **Drug Discovery and Materials Science:**  Simulating the behavior of molecules to design new drugs and materials.
    *   **Cryptography:**  Breaking existing encryption algorithms (a potential threat) and developing new, quantum-resistant ones.
    *   **Optimization:**  Finding the best solution to complex optimization problems, like logistics, finance, and machine learning.
    *   **Machine Learning:**  Developing new and more powerful machine learning algorithms.

*   **Important Note:** Quantum computers are *not* going to replace your laptop or phone. They are specialized tools for solving specific types of problems that are currently intractable for classical computers. Also, quantum computers are still in early stages of development.

**In summary:**

Classical computers: Like light switches (0 or 1).  Do things one step at a time.
Quantum computers: Like dimmer switches (0, 1, or anything in between *at the same time*). Can explore many possibilities simultaneously.

Think of it this way: Finding your way out of a maze.

*   **Classical Computer:** Tries each path one by one until it finds the exit.
*   **Quantum Computer:** Explores all possible paths *simultaneously* to find the exit much faster.

Hope this helps! Let me know if you have any other questions.


In [9]:
view = gr.Interface(
    fn=stream_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=gr.Markdown(),
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


### Chat Bot With History

In [ ]:
def stream_gemini_with_history(message, history):
        
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )
    
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {message}\nAssistant:"

    stream = model.generate_content(prompt, stream=True)

    response = ""
    for chunk in stream:
        if chunk.text:
            response += chunk.text
            yield history + [(message, response)], ""


In [13]:
def clear_chat():
    return [], ""

with gr.Blocks(theme=gr.themes.Soft()) as view:
    gr.Markdown("## 💬 Chat Bot with History")

    chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)
    msg = gr.Textbox(label="Your message:", lines=2, placeholder="Type your message here...")

    with gr.Row():
        send_btn = gr.Button("Send")
        clear_btn = gr.Button("Clear Chat")

    msg.submit(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])
    send_btn.click(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])

    clear_btn.click(clear_chat, outputs=[chatbot, msg])

view.launch()


C:\Users\chandula\AppData\Local\Temp\ipykernel_15184\1344445376.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


[]
[['hi', 'Hi there! How can I help you today?\n']]
[['hi', 'Hi there! How can I help you today?\n'], ['what are acid\n', "Acids are substances that typically have the following properties:\n\n*   **Taste:** Sour (though you should NEVER taste an unknown substance to determine if it's an acid!).\n*   **Reaction with Metals:** They can react with some metals to release hydrogen gas.\n*   **Litmus Paper:** They turn blue litmus paper red.\n*   **pH:** They have a pH less than 7.\n*   **Neutralization:** They can neutralize bases (alkalis).\n*   **Chemical Definition:** More technically, an acid is a molecule or ion capable of donating a proton (hydrogen ion, H+) or forming a covalent bond with an electron pair.\n\nCommon examples of acids include:\n\n*   **Hydrochloric acid (HCl):** Found in gastric acid in the stomach.\n*   **Sulfuric acid (H2SO4):** Used in many industrial processes.\n*   **Acetic acid (CH3COOH):** Found in vinegar.\n*   **Citric acid (C6H8O7):** Found in citrus fruit